## Figuring out Spark 2.0

aka the *Decimal Project*

### Notes/Discoveries
an empty core-site.xml causes 
```
---> 18 sc = SparkContext(conf=conf)
Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
 : java.lang.ExceptionInInitializerError
```

In [1]:
##### CELL A #####

# Basic setup required in all notebooks
 
appName='archetest'
master='local[*]' #local spark-master

# Explicitly define python 2 since we have both 2 & 3 installed
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python'

# Establish spark context or session
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)


# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

[432, 207, 356, 638, 47]

In [2]:
##### CELL B #####

# read local file
localFilePath = "/home/jovyan/work/fotd.csv"
d = sc.textFile(localFilePath)
print type(d)
d.take(2)

<class 'pyspark.rdd.RDD'>


[u'sheet,Date,Event,Lesson,attendees_reg,attendees_student,attendees_special,attendees_board,addendees_free,addendees_coupon,addendees_comp,fee_reg,fee_student,fee_special,fee_board,donations,caller,band',
 u'4/16/2016,4/16/2016,Contra Dance Financial Worksheet,-1,37,11,0,4,6,0,6,$9.00,$5.00,$8.00,$9.00,$37.00,Patricia Dancen,Steam']

In [3]:
##### CELL C #####

#import hadoop
def set_hadoop_config():
    
    # Turns out setting parameters via hadoopConfiguration is (apparently) not useful for spark 2.0
    
    #Required parameter names listed in http://spark.apache.org/docs/latest/storage-openstack-swift.html

    prefix = "fs.hdfs.service." + appName  #some refs use creds['name'], which can be any string
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + ".auth.url", r'http://172.18.0.2' )
    hconf.set(prefix + ".username", 'root')
    hconf.set(prefix + ".http.port", '9000')
    hconf.setBoolean(prefix + ".public", True)

    # TRY: download jars & add to Dockerfile
    #org.apache.hadoop.
    #hconf.set("fs.hdfs.impl", hdfs.DistributedFileSystem.getName());
    #hconf.set("fs.file.impl", org.apache.hadoop.fs.LocalFileSystem.getName());
    #hconf.set("fs.swift.impl", "org.apache.hadoop.fs.swift.snative.SwiftNativeFileSystem")


In [4]:
##### CELL D #####

# Define the datafile and establish access, spark 2

#set_hadoop_config()  # not helpful!

datafileHDFS = r'hdfs://172.18.0.2:9000/user/root/testData/fotd.csv'
d = sc.textFile(datafileHDFS)
print type(d)
d.take(2)


<class 'pyspark.rdd.RDD'>


[u'sheet,Date,Event,Lesson,attendees_reg,attendees_student,attendees_special,attendees_board,addendees_free,addendees_coupon,addendees_comp,fee_reg,fee_student,fee_special,fee_board,donations,caller,band',
 u'4/16/2016,4/16/2016,Contra Dance Financial Worksheet,-1,37,11,0,4,6,0,6,$9.00,$5.00,$8.00,$9.00,$37.00,Patricia Dancen,Steam']

In [6]:
stop here  # halt execution with an error


#this is stuff I might yet use

from __future__ import print_function
import sys
from pyspark.sql import SparkSession

SyntaxError: invalid syntax (<ipython-input-6-b71a775f3f87>, line 1)

In [5]:
%env

{'APACHE_SPARK_VERSION': '2.0.0',
 'CLICOLOR': '1',
 'CONDA_DIR': '/opt/conda',
 'DEBIAN_FRONTEND': 'noninteractive',
 'GIT_PAGER': 'cat',
 'GRANT_SUDO': 'yes',
 'HOME': '/home/jovyan',
 'HOSTNAME': '93ad4a639d79',
 'JPY_PARENT_PID': '9',
 'LANG': 'en_US.UTF-8',
 'LANGUAGE': 'en_US.UTF-8',
 'LC_ALL': 'en_US.UTF-8',
 'LOGNAME': 'jovyan',
 'MAIL': '/var/mail/jovyan',
 'MESOS_NATIVE_LIBRARY': '/usr/local/lib/libmesos.so',
 'NB_UID': '1000',
 'NB_USER': 'jovyan',
 'PAGER': 'cat',
 'PATH': '/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
 'PWD': '/home/jovyan/work',
 'PYSPARK_PYTHON': '/opt/conda/envs/python2/bin/python',
 'PYTHONPATH': '/usr/local/spark/python:/usr/local/spark/python/lib/py4j-0.10.1-src.zip',
 'R_LIBS_USER': '/usr/local/spark/R/lib',
 'SHELL': '/bin/bash',
 'SHLVL': '1',
 'SPARK_HOME': '/usr/local/spark',
 'SPARK_OPTS': '--driver-java-options=-Xms1024M --driver-java-options=-Xmx4096M --driver-java-options=-Dlog4j.logLevel=info',
 'TERM': 'xter

In [ ]:
stop here

In [ ]:
#once I have the data, do something with it

dataRDD2 = dataRDD.map(lambda x: x.split('\r\n'))
print type(dataRDD2), dataRDD2
print dataRDD2.count() 
prSomeRDD(dataRDD2,1) 

In [ ]:
# Handy methods


def prSomeRDD(inputRDD,howMany,msg=None):
    """
    Debugging shorthand

    Args:

    Returns: None
        
    """
    print msg
    #inputRDD.take(howMany).foreach(println)
    for x in inputRDD.take(howMany):
        print x    
